Sentence-Transformers

  - BERT 계열 모델의 토큰 출력을 Pooling하여 고정 크기 문장 벡터를 생성하는 시스템
  - 두 가지 인코딩 방식
    - **Sentence Encoder (Bi-Encoder)**
      - 각 문장을 독립적으로 인코딩
      - 빠름: O(n)
      - 벡터끼리 코사인 유사도만 계산하면 됨
      - Sentence-BERT가 사용하는 방식
      - 용도: 검색, 유사도 비교, 군집화
    - **Cross-Encoder**
      - 문장 2개를 동시에 BERT에 넣어 분류”하는 구조
      - 느림: O(n²)
      - 정확도는 높음

Pooling 종류 (Sentence-BERT의 핵심 단계) 
- Pooling의 역할 — “토큰 벡터를 문장 벡터로 압축하는 과정”
  1) [CLS] Pooling  
  첫 번째 토큰([CLS])만 사용  
  성능: 보통  

  2) Mean Pooling  
  모든 토큰의 임베딩 평균 → 문장 벡터  
  성능: 가장 좋음 (SBERT 기본 추천)  

  3) Max Pooling  
  각 차원별로 최대값을 선택  
  성능: 보통  

한국어 모델 활용(KR-SBERT)
- 한국어 데이터로 파인튜닝된 Sentence-BERT 모델
- KorSBERT / KoSentenceBERT 등
- 한국어 유사도 평가나 문장 검색에 효과적

In [ ]:
# Sentence Transformer
# 1) 모델 로드
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 2) 문장 목록과 임베딩
sentences = [
    '오늘 날씨가 좋아요',
    '오늘 하늘이 맑아요',
    '프로그램을 배우고 싶습니다.'
]
embeddings = model.encode(sentences)       
# shape -> (3, D)/ 문장을 의미 기반의 벡터 표현으로 바꾸는 표정
# 문장을 토큰화/ 토큰을 BERT로 인코딩/ 토큰 임베딩을 문장 임베딩함
# BERT로 인코딩 -> 문맥을 이해한 벡터로 변환한다는 뜻 -> 입력 문장을 토큰으로 쪼개고 앞뒤 문장을 고려해서 각 토큰을 숫자벡터로 바꾸는 과정
# => 단순 숫자가 아닌 문장의 의미 관계 문맥을 반영한 표현으로 바꾸는 것 
print(f'임베딩 크기 : {embeddings.shape}')

# 3) 유사도 계산 (코사인 유사도) -> 벡터 방향 비교
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(embeddings)  # 모든 문장 쌍을 서로 비교
print(f'유사도 행렬 : {similarity}')


임베딩 크기 : (3, 768)
유사도 행렬 : [[1.         0.80070263 0.24432442]
 [0.80070263 1.         0.178646  ]
 [0.24432442 0.178646   1.        ]]


In [ ]:
from sentence_transformers import SentenceTransformer, models 
transformer = models.Transformer('klue/roberta-base')
# 레이어 추가
pooling = models.Pooling(
    transformer.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,  # mean pooling 사용
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
# 모델 조립
model = SentenceTransformer(modules=[transformer, pooling])
# 문장 임베딩 생성
sentence = ['안녕하세요','반갑습니다']
embeddings = model.encode(sentences)
print(embeddings)